In [1]:
import pandas as pd
import numpy as np
import zipfile

from functions.load import *

In [6]:
import country_converter as coco

def to_iso(series, target):
    return coco.convert(series, to=target, not_found=np.nan)

def to_region(series, scheme):
    out = coco.convert(series, to=scheme, not_found=None)
    return pd.Series(out, index=series.index, name=scheme)

def adjust_cols(df, country, year, iso_code_1=None, iso_code_2=None, region=None):
    df = df.sort_values(by=country).reset_index(drop=True)
    df = df.rename(columns={country: 'country', year: 'year'})
    df = df.drop(columns=[c for c in (iso_code_1, iso_code_2, region) if c], errors='ignore')
    df['iso_code'] = to_iso(df['country'], 'ISO3')
    df['region'] = to_region(df['country'], "UNregion")
    df = df[['country', 'year', 'iso_code', 'region'] + [c for c in df.columns if c not in {'country', 'year', 'iso_code', 'region'}]]
    df = df.sort_values(by=['country', 'year']).reset_index(drop=True)
    return df

countries = pd.Series(["Germany","Côte d’Ivoire","Fiji"])
to_region(countries, 'UNregion')

In [2]:
datasets = load()

(efw, # iso 1,2; 1970-2021
 fiw, # no iso; 2006-2023
 ief, # iso 1; 1995-2023
 p5d, # no iso; 1960-2020
 pts, # iso 2; 1976-2022
 wgi, # iso 2; 1996-2022
 wb # iso 2; 1960-2023
) = (datasets[k] for k in ["efw", "fiw", "ief", "p5d", "pts", "wgi", "wb"])

Loading pickles/EFW.xlsx
Loading pickles/FIW.xlsx
Loading pickles/IEF.csv
Loading pickles/P5D.xls
Loading pickles/PTS.xlsx
Loading pickles/WGI.xlsx
Loading pickles/WB_DATA.zip


In [ ]:
# create concordance for all unique countries in datasets

In [ ]:
from keras.layers import LSTM
from tensorflow.keras import datasets, layers, models
from tensorflow import keras

model = keras.Sequential([
    keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    keras.layers.LSTM(50, return_sequences=False),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(17, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=300, batch_size=100, validation_split=0.1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

2025-02-09 18:03:38.552487: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 18:03:38.918876: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 18:03:39.439476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-09 18:03:47.940592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/300


2025-02-09 18:04:01.210472: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 8913509640 exceeds 10% of free system memory.


5054/5054 ━━━━━━━━━━━━━━━━━━━━ 32s 6ms/step - accuracy: 0.6992 - loss: 0.7166 - val_accuracy: 0.8665 - val_loss: 0.3221
Epoch 2/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.8850 - loss: 0.2845 - val_accuracy: 0.9122 - val_loss: 0.2136
Epoch 3/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9190 - loss: 0.2046 - val_accuracy: 0.9353 - val_loss: 0.1735
Epoch 4/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9363 - loss: 0.1668 - val_accuracy: 0.9410 - val_loss: 0.1531
Epoch 5/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9445 - loss: 0.1478 - val_accuracy: 0.9492 - val_loss: 0.1319
Epoch 6/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9498 - loss: 0.1339 - val_accuracy: 0.9525 - val_loss: 0.1288
Epoch 7/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9549 - loss: 0.1231 - val_accuracy: 0.9590 - val_loss: 0.1166
Epoch 8/300
5054/5054 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.9578 - loss: 0.116